# <h1 style="text-align:center;"> Introduction </h1>
The purpose of this notebook is to download and condense data from different sources into a single Dataframe, saving it to a csv file, to later use when building our predictive models.

# <h1 style="text-align:center;"> Imports </h1>

In [1]:
import pandas as pd
import requests
import re
import json
import matplotlib as plt
from tqdm import tqdm
import sys
import io
import numpy as np

# <h1 style="text-align:center;"> Downloading S&P500 stock tickers </h1>

Due to this procedure relying on Wikipedia as a source, we will store the result in a csv file, that way, if the layout of the page changes we can still use the data.

In [2]:
def get_sp500_stocks():
  stocks=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
  res = []
  for stock in stocks:
    res.append(stock)
  return res

In [3]:
print(get_sp500_stocks())

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'C

# List of Ratios to Download

In [4]:
#TODO: Annotate all the values, and wether they are %, absolute, TTM, also add YoY for some (like gross-profit)

price_ratios = [                      #Only v3
    'pe-ratio',
    'price-sales',
    'price-book',
    'price-fcf',
    'current-ratio',
    'quick-ratio',
    'debt-equity-ratio',
    'roe',
    'roa',
    'return-on-tangible-equity',
    'ebitda-margin',
    ]

balance_sheet = [                       #v1 quarterly, v2 YoY
    'total-assets',                    
    'total-liabilities',                
    'total-share-holder-equity',
    'long-term-debt',
    'shares-outstanding'
    ]

income_statement = [                    # YoY and TTM for all of these v1: TTM, v2: Quarterly, v3: YoY
    'eps-earnings-per-share-diluted',   #Possible YoY
    'net-income',                       #Possible YoY
    'ebitda',                           #Possible YoY
    'operating-income',                 #Possible YoY
    'gross-profit',                     #Possible YoY
    'revenue',                          #Possible YoY
    'shares-outstanding',               #Special case, v1: quarterly, v2: YoY
]

In [5]:
def fetch_special_cases(stock):
    div_url = f"https://www.macrotrends.net/assets/php/dividend_yield.php?t={stock}"
    num_emply_url = f"https://www.macrotrends.net/assets/php/fundamental_iframe.php?t={stock}&type=number-of-employees&statement=&freq=A"
    profit_margins_url = f"https://www.macrotrends.net/assets/php/fundamental_metric.php?t={stock}&chart=profit-margin"
   
    #Profit Margins
    data = []
    html_doc = requests.get(profit_margins_url).text
    res = re.search(r"chartData = (\[\{.*?\}\])", html_doc)
    if res is None:
        return None
    res = res.group(1)
    res = json.loads(res)
    for entry in res:
        data.append((entry['date'], stock, entry['v1'], entry['v2'], entry['v3']))
    df = pd.DataFrame(data, columns=['Date', 'Ticker', 'ttm-gross-margin', 'ttm-operating-margin', 'ttm-net-margin'])


    #Number of Employees
    data = []
    html_doc = requests.get(num_emply_url).text
    res = re.search(r"chartData = (\[\{.*?\}\])", html_doc)
    if res is None:
        return None
    res = res.group(1)
    res = json.loads(res)
    for entry in res:
        data.append((entry['date'], stock, entry['v2']))
    df_emply = pd.DataFrame(data, columns=['Date', 'Ticker', 'number-employees'])
    df = pd.merge(df, df_emply, how='outer')
    df["number-employees"].ffill(inplace=True)
    return 

    

In [6]:
def fetch_ratio(ratio, stock_symbol, l_entries, section, suffixes=[],):
  data = []
  url = f"https://www.macrotrends.net/assets/php/fundamental_iframe.php?t={stock_symbol}&type={ratio}&statement={section}&freq=Q"
  entries = ()
  columns = ['Date', 'Ticker']
  if len(suffixes) == 0:
    columns.append(ratio)
  else:
    for suffix in suffixes:
      columns.append(f"{ratio}_{suffix}")

  html_doc = requests.get(url).text
  res = re.search(r"chartData = (\[\{.*?\}\])", html_doc)
  if res is None:
    return None
  res = res.group(1)
  res = json.loads(res)
  for entry in res:
    entries = (entry['date'], stock_symbol, )
    for v_i in l_entries:
      entries += (entry[v_i],)
    data.append(entries)

  return pd.DataFrame(data, columns=columns)

In [7]:
def fetch_ratios(stock_symbol):
    df = pd.DataFrame()
    for ratio in price_ratios:
        df_ratio = fetch_ratio(ratio, stock_symbol, ['v3'], 'price-ratios')
        if df_ratio is not None:
            if df.empty:
                df = df_ratio
            else:
                df = pd.merge(df, df_ratio, how='outer')
    for ratio in balance_sheet:
        df_ratio = fetch_ratio(ratio, stock_symbol, ['v1','v2'], 'balance-sheet', ['Q','YoY'])
        if df_ratio is not None:
            df = pd.merge(df, df_ratio, how='outer')
    for ratio in income_statement:
        if ratio == 'shares-outstanding':
            df_ratio = fetch_ratio(ratio, stock_symbol, ['v1', 'v2'], 'income-statement', ['Q', 'YoY'])
        else:
            df_ratio = fetch_ratio(ratio, stock_symbol, ['v1', 'v2', 'v3'], 'income-statement', ['TTM', 'Q', 'YoY'])
        if df_ratio is not None:
            df = pd.merge(df, df_ratio, how='outer')
    
    df_spec = fetch_special_cases(stock_symbol)
    if df_spec is not None:
        df = pd.merge(df, df_spec, how='left')
    return df    

In [8]:
def fetch_yahoo(stock_symbol):
  headers = {
      'User-Agent': 'Mozilla/5.0'
  }
  url = "https://query1.finance.yahoo.com/v7/finance/download/"
  url += str(stock_symbol)
  url += "?period1=1030060800&period2=1684368000&interval=1d&events=history&includeAdjustedClose=true"
  r = requests.get(url, headers=headers)
  pds = pd.read_csv(io.StringIO(r.text), index_col=0, parse_dates=True)
  return pds

In [103]:
fetch_yahoo("AAPL").index.inferred_type

'datetime64'

In [9]:
def fetch_dataframes(stock_symbol):
    #Extracting dataframe with prices for the stock
    if stock_symbol == 'BF.B':
        df_prices = fetch_yahoo("BF-B")
    elif stock_symbol == 'BRK.B':
        df_prices = fetch_yahoo("BRK-B")
    else:
        df_prices = fetch_yahoo(stock_symbol)
    df_prices.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)
    #Fill all dates
    start_date = str(df_prices.index[0])
    end_date = str(df_prices.index[-1])
    
    idx = pd.date_range(start_date, end_date)
    df_prices = df_prices.reindex(idx)
    df_prices.ffill(inplace=True)
    df_prices.index = df_prices.index.astype("string")

    #Extracting dataframe with ratios and metrics for the stock
    df_ratios = fetch_ratios(stock_symbol)
    df_ratios.set_index('Date', inplace=True)
    df_ratios.sort_index(inplace=True)
    df_ratios.index.name=None

    return df_prices, df_ratios

In [10]:
def fetch_data(stock_symbol):
    df_p, df_r = fetch_dataframes(stock_symbol)
    df = df_r.merge(df_p, how='left', left_index=True, right_index=True)
    #Now we create the column of the annual change
    df['Yearly Change (%)'] = (df['Adj Close'].shift(-4) - df['Adj Close']) / (df['Adj Close']) * 100.0
    return df

In [12]:
def create_dataset():
    #Fetch SP500 stock ticker list
    list_sp500 = get_sp500_stocks()
    df = pd.DataFrame()
    f = open('logging.txt', 'w')
    #sys.stdout = f # Change the standard output to the file we created.

    for stock in tqdm(list_sp500):
        df_stock = fetch_data(stock)
        df = pd.concat([df, df_stock])
    
    return df.to_csv('data/sp500_quarterly_data.csv')

In [13]:
create_dataset()

100%|██████████| 503/503 [1:18:27<00:00,  9.36s/it]


In [178]:
df = pd.read_csv('data/sp500_quarterly_data.csv', index_col=0)

In [19]:
df = df[df.isnull().sum(axis=1) < 20]

# <h1 style="text-align:center;"> Adding Macroeconomic Variables  </h1> 
The variables we added before determine the fundamental analysis of a company, from a quantitative point of view. However, macroeconomic factors cannot be ignored when assesing the health of a company and it's future returns. These have to be incorporated into the portfolio management strategy of any investor, an thus, we will do the same in our dataset. We will incorporate the following metrics:
* **Economic growth** - There's a discussion to be had to determine what metric best encapsules economic growth, we will be using <span style="color:blue">GDP at purchaser's prices</span>.
* **Unemployment Rate**.
* **Inflation**.
* **Interest Rates** - For this, we will use the <span style="color:blue">federal funds rates</span>.
Many more variables could be included, and each one of them is worthy of a study on it's own, but for the meantime we will work with these metrics. They will all be taken from macrotrends.net, as we did for the ratios.

In [123]:
macros = [
    ("https://www.macrotrends.net/countries/USA/united-states/economic-growth-rate", 'growth-rate', 'v3'),
    ("https://www.macrotrends.net/countries/USA/united-states/unemployment-rate", 'unemployment-rate', 'v1'),
    ("https://www.macrotrends.net/countries/USA/united-states/inflation-rate-cpi", 'inflation-rate', 'v1'),
]

In [124]:
df_rates = pd.read_csv('data/fed-funds-rate-historical-chart.csv')
df_rates.head(5)

,Date,interest-rates
0,1954-07-01,1.13
1,1954-07-02,1.25
2,1954-07-03,1.25
3,1954-07-04,1.25
4,1954-07-05,0.88


In [125]:
def fetch_macro(url, macro, num_entry):
  html_doc = requests.get(url).text
  res = re.search(r"chartData = (\[\{.*?\}\])", html_doc)
  if res is None:
    return None
  res = res.group(1)
  res = json.loads(res)
  data = []
  for entry in res:
    if entry['date'] >= "2008-01-01":
      data.append((entry['date'], entry[num_entry]))
  return pd.DataFrame(data, columns=['Date', macro])

In [165]:
def fetch_macros():
    df = pd.DataFrame()
    for macro in macros:
        df_macro = fetch_macro(macro[0], macro[1], macro[2])
        if df_macro is not None:
            if df.empty:
                df = df_macro
            else:
                df = pd.merge(df, df_macro, how='outer')
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    
    df_rates = pd.read_csv('fed-funds-rate-historical-chart.csv')
    df_rates.set_index('Date', inplace=True)
    df_rates.sort_index(inplace=True)
    df_rates = df_rates[df_rates.index >= "2008-01-01"]

    df = pd.merge(df, df_rates, how='outer', left_index=True, right_index=True)
    df.index = pd.to_datetime(df.index)
    start_date = df.index[0]
    end_date = df.index[-1]
    idx = pd.date_range(pd.to_datetime(start_date), pd.to_datetime(end_date))
    df = df.reindex(idx)
    df.ffill(axis=0, inplace=True)
    df = df[df.index.year >= 2009]
    return df

In [174]:
df_macros = fetch_macros()
df.index = pd.to_datetime(df.index)
df = pd.merge(df, df_macros, how='left', left_index=True, right_index=True)
df.sample(5)

datetime64
datetime64


,Ticker,pe-ratio,price-sales,price-book,price-fcf,current-ratio,quick-ratio,debt-equity-ratio,roe,roa,...,gross-profit_YoY,revenue_TTM,revenue_Q,revenue_YoY,Adj Close,Yearly Change (%),growth-rate,unemployment-rate,inflation-rate,interest-rates
2016-12-31,VTRS,39.32,1.69,1.74,11.36,1.492,1.005,1.490,4.41,1.56,...,25.59,11.077,3.268,31.19,35.381607,10.904321,1.75,4.87,1.2616,0.55
2011-12-31,VRSN,58.56,7.97,0.00,34.33,2.197,NaN,-7.832,200.00,7.49,...,14.89,0.773,0.204,13.97,35.720001,8.678608,4.64,8.95,3.1568,0.04
2013-09-30,VTRS,23.97,2.12,4.28,24.80,1.696,1.053,2.105,19.52,4.95,...,2.02,6.824,1.767,-1.94,35.400150,19.177393,3.13,8.07,2.0693,0.06
2013-01-31,BF.B,21.13,4.39,8.02,19.69,2.535,1.542,0.656,29.00,15.81,...,12.42,2.793,0.747,6.41,22.045500,20.828146,0.46,8.07,2.0693,0.15
2020-12-31,MMM,17.12,2.90,7.16,13.64,1.885,1.352,1.391,47.38,11.84,...,10.80,32.184,8.583,5.82,157.996124,4.850570,2.56,8.05,1.2336,0.09


## <h2 style="text-align:center;"> Saving the final dataframe to a file </h2>

In [175]:
df.to_csv('data/sp500_macro_quarterly_data.csv')